In [ ]:
#@title Install PyDDM and download the script containing the models
!pip -q install git+https://github.com/mwshinn/PyDDM
import hashlib
import requests
import os

fname = "shinn2020.py"
url = "https://raw.githubusercontent.com/mwshinn/PyDDM/master/doc/downloads/shinn2020.py"

if not os.path.isfile(fname):
    r = requests.get(url)
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [ ]:
#@title Visualize the model from Shinn et al (2020)
try:
    from ddm import Model, Fitted, OverlayNone, OverlayPoissonMixture, OverlayChain, OverlayNonDecision
    import ddm.plot
    from shinn2020 import DriftShinn2020, NoiseShinn2020, BoundCollapsingExponentialDelay, ICPoint, OverlayMappingError
except ImportError:
    raise ImportError("Please run the previous cell to install PyDDM and download the script before trying to visualize the model")
m = Model(name='Collapsing bounds model from Shinn et al (2020)', 
          drift=DriftShinn2020(snr=Fitted(9.318192711041897, minval=0.5, maxval=40), 
                               noise=Fitted(1.026120377999896, minval=0.01, maxval=4), 
                               t1=0, t1slope=0, cohexp=1, maxcoh=70, 
                               leak=Fitted(7.1429314579797385, minval=0.01, maxval=40), 
                               leaktarget=Fitted(0.08630984715844775, minval=0, maxval=0.9)), 
          noise=NoiseShinn2020(noise=Fitted(1.026120377999896, minval=0.01, maxval=4), t1=0, t1slope=0), 
          bound=BoundCollapsingExponentialDelay(B=1, 
                                                tau=Fitted(1.1859413319935954, minval=0.1, maxval=10), 
                                            t1=Fitted(0.3611682749713456, minval=0, maxval=1)), 
          IC=ICPoint(x0=Fitted(0.08630984715844775, minval=0, maxval=0.9)), 
          overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fitted(0.21768400393149134, minval=0.1, maxval=0.3)), 
                                         OverlayMappingError(mappingcoef=Fitted(0.11815235699328804, minval=0, maxval=0.2)), 
                                         OverlayPoissonMixture(pmixturecoef=Fitted(0.042431596838114934, minval=0.001, maxval=0.1), 
                                                               rate=Fitted(1.0219359822883225, minval=0.1, maxval=2))]), 
          dx=0.01, dt=0.01, T_dur=3.0)


ddm.plot.model_gui_jupyter(m, conditions={"coherence": [50, 53, 60, 70],
                                          "presample": [0, 400, 800],
                                          "highreward": [0, 1]})